In [1]:
import sys
sys.path.append('/home/student/project/HumanChoicePrediction/')

In [4]:
import pandas as pd
from consts import N_HOTELS
import glob
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import opinion_lexicon
from nltk.tokenize import treebank
import numpy as np
# nltk.download('vader_lexicon')
# nltk.download('opinion_lexicon')
# nltk.download('punkt')

# pd.options.mode.chained_assignment = None

In [20]:
pd.set_option('display.max_colwidth', None)
# pd.reset_option('display.max_colwidth')

In [21]:
df['concat_review']

0                                                                                                                                                                                                                                                                                Very nice room. Not too much choice for breakfast and the swimming pool is not included
1                                                                                                               Top facilities. Heating didn't work. It was warmer after I switched it off than before. Also breakfast was average for a hotel of such stature. Although, if I didn't ask I wouldn't have known that you can order an omlette for yourself at breakfast.
2                                                                                                                                                                                                                                                                     nan. Badly maint

In [5]:
df = pd.read_csv('combined_reviews_with_sentiment_scores.csv')
df

,ID,positive_review,negative_review,rating,concat_review,positive_score,negative_score,positive_to_negative_sentiment_ratio,is_good_review,positive_review_length,negative_review_length,positive_to_negative_length_ratio,review_length
0,2411,Very nice room,Not too much choice for breakfast and the swim...,6.0,Very nice room. Not too much choice for breakf...,0.171,0.000,100.000000,0,3,13,0.230769,17
1,2272,Top facilities,Heating didn't work. It was warmer after I swi...,8.3,Top facilities. Heating didn't work. It was wa...,0.095,0.000,100.000000,1,2,50,0.040000,53
2,2561,NaN,Badly maintained hotel many switches not funct...,6.3,nan. Badly maintained hotel many switches not ...,0.000,0.242,0.000000,0,1,18,0.055556,20
3,1321,"Beautiful little balconette, gorgeous bathroom...","A few little stains, no coffee machine and emp...",8.0,"Beautiful little balconette, gorgeous bathroom...",0.355,0.116,3.060345,1,15,13,1.153846,28
4,1639,"Everything was good, location was good for sho...",Everything was comforting,10.0,"Everything was good, location was good for sho...",0.451,0.000,100.000000,1,25,3,8.333333,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7471,23647,"Location, room size, parking, staff",Noise from the highway with windows open.,9.0,"Location, room size, parking, staff. Noise fro...",0.000,0.000,0.000000,1,8,8,1.000000,17
7472,23652,The Locations is good.\nGood breakfast in COVI...,Very small room for a twin bed room.\nPoor ref...,6.0,The Locations is good.\nGood breakfast in COVI...,0.221,0.168,1.315476,0,12,23,0.521739,36
7473,24232,Parking space outside the hotel,"When I arrived to check in nobody was there, I...",4.6,Parking space outside the hotel. When I arrive...,0.000,0.126,0.000000,0,5,64,0.078125,70
7474,23728,The staff were polite and proffesional.,The breakfast could be more fresh and should c...,7.0,The staff were polite and proffesional.. The b...,0.166,0.143,1.160839,0,7,65,0.107692,72


In [10]:
%env OPENAI_API_KEY="sk-XXXXXXXXX"

None


In [9]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [8]:
location_schema = ResponseSchema(name="location",
                             description="You are reading an hotel review.\
                             Does the review mentions the location of the hotel? if yes, output 1. if not output 0.")
food_schema = ResponseSchema(name="food",
                                      description="You are reading an hotel review.\
                                      Does the review mentions the food of the hotel? if yes, output 1. if not output 0.")
service_schema = ResponseSchema(name="service",
                                    description="You are reading an hotel review.\
                                    Does the review mentions the service of the hotel? if yes, output 1. if not output 0.")

comfort_schema = ResponseSchema(name="comfort",
                                      description="You are reading an hotel review.\
                                      Does the review mentions the comfort of the hotel? if yes, output 1. if not output 0.")
cleanliness_schema = ResponseSchema(name="cleanliness",
                                    description="You are reading an hotel review.\
                                    Does the review mentions the cleanliness of the hotel? if yes, output 1. if not output 0.")

response_schemas = [location_schema, 
                    food_schema,
                    service_schema,
                   comfort_schema,
                    cleanliness_schema]

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [13]:
customer_review = "Top facilities. Heating didn't work. It was warmer after I switched it off than before. Also breakfast was average for a hotel of such stature. Although, if I didn't ask I wouldn't have known that you can order an omlette for yourself at breakfast."

In [ ]:
review_template = """\
For the following text, extract the following information:

location: Does the review mentions the location of the hotel? if yes, output 1. if not output 0.
food: Does the review mentions the food of the hotel? if yes, output 1. if not output 0.
service: Does the review mentions the service of the hotel? if yes, output 1. if not output 0.
comfort: Does the review mentions the comfort of the hotel? if yes, output 1. if not output 0.
cleanliness: Does the review mentions the cleanliness of the hotel? if yes, output 1. if not output 0.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)